In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers, models, callbacks
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score
from fairlearn.metrics import equalized_odds_difference, equalized_odds_ratio

In [5]:
read_dir = './pipeline/cleaned/'

cohort = pd.read_csv(read_dir + 'cohort.csv', index_col=0)

diag = pd.read_csv(read_dir + 'diag.csv', index_col=0)
diag = diag.set_index('hadm_id')

labs = pd.read_csv(read_dir + 'labs.csv', index_col=0)
labs = labs.set_index('hadm_id')

proc = pd.read_csv(read_dir + 'proc.csv', index_col=0)
proc = proc.set_index('hadm_id')

meds = pd.read_csv(read_dir + 'meds.csv', index_col=0)
meds = meds.set_index('hadm_id')

In [7]:
def initialize(lr, cohort, diag, labs, proc, meds, use_diag, use_labs, use_proc, use_meds, use_insurance, use_gender, use_race, hf):
    if hf:
        split = 'split_hf'
    else:
        split = 'split_diabetes'
    train_hadm = cohort[cohort[split]=='train'].hadm_id.to_numpy()
    valid_hadm = cohort[cohort[split]=='validation'].hadm_id.to_numpy()
    test_hadm = cohort[cohort[split]=='test'].hadm_id.to_numpy()
    train_hadm = np.sort(train_hadm)
    valid_hadm = np.sort(valid_hadm)
    test_hadm = np.sort(test_hadm)
    model = Transformer(lr, cohort, diag, labs, proc, meds, use_diag, use_labs, use_proc, use_meds, use_insurance, use_gender, use_race, hf)
    print(model.model.summary())
    return model, train_hadm, valid_hadm, test_hadm


class Transformer(object):
    def __init__(self, lr, cohort, diag, labs, proc, meds, use_diag, use_labs, use_proc, use_meds, use_insurance, use_gender, use_race, hf):
        keras.backend.clear_session()
        physical_devices = tf.config.experimental.list_physical_devices('GPU')
        #print(f"GPU list {physical_devices}")
        #tf.config.experimental.set_memory_growth(physical_devices[0], True)
        self.opt = keras.optimizers.Adam(learning_rate=lr)
        self.loss = keras.losses.BinaryCrossentropy()
        self.metric = keras.metrics.AUC(name='auc')
        
        self.cohort = cohort
        self.diag = diag.copy()
        self.labs = labs
        self.proc = proc
        self.meds = meds
        
        self.use_diag = use_diag
        self.use_labs = use_labs
        self.use_proc = use_proc
        self.use_meds = use_meds
        self.use_insurance = use_insurance
        self.use_gender = use_gender
        self.use_race = use_race
        
        self.input_shapes = self.get_input_shapes()
        if hf:
            self.label = 'label_hf'
            self.diag['I50'] = 0
        else:
            self.label = 'label_diabetes'
            self.diag['E11'] = 0
         
        self.buildModel()
    
    def embedding(self, inputs):
        maxlen = inputs.shape[-2]
        varnum = inputs.shape[-1]
        pos = keras.backend.arange(start=0, stop=maxlen, step=1)
        pos = layers.Embedding(input_dim=maxlen, output_dim=256)(pos)
        x = layers.Dense(units=256)(inputs)
        return x + pos
    
    def encoder(self, inputs):
        hidden_dim = inputs.shape[-1]
        if len(inputs.shape) == 2:
            y = layers.Dense(hidden_dim)(inputs)
            y = layers.Dropout(0.1)(y)
            y = layers.LayerNormalization(epsilon=1e-6)(y)
            return tf.expand_dims(y, 1)
        
        x = layers.MultiHeadAttention(num_heads=8, key_dim=hidden_dim)(inputs, inputs)
        x = layers.Dropout(0.1)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)
        
        y = layers.Dense(hidden_dim, activation='relu')(x)
        y = layers.Dense(hidden_dim)(y)
        y = layers.Dropout(0.1)(y)
        y = layers.LayerNormalization(epsilon=1e-6)(y + x)
        return y
    
    def buildModel(self): 
        inputs = [keras.Input(shape=shape) for shape in self.input_shapes]
        encoded = [self.encoder(inp) for inp in inputs]
        if len(inputs) > 1:
            combined = layers.Concatenate(axis=-1)([layers.GlobalAveragePooling1D()(enc) for enc in encoded])
        else:
            combined = layers.GlobalAveragePooling1D()(encoded[0])
        x = layers.Dense(64, activation="relu")(combined)
        outputs = layers.Dense(1, activation="sigmoid")(x)

        self.model = keras.Model(inputs=inputs, outputs=outputs)
        
        self.model.compile(optimizer=self.opt, loss=self.loss, metrics=[self.metric])
    
    def train(self, train_hadm, valid_hadm, epochs, batchSize, path, val_auc=False, verbose_ckp=1, verbose_fit=2):
        if val_auc:
            MCP = callbacks.ModelCheckpoint(path, monitor='val_auc', verbose=verbose_ckp, save_best_only=True, mode='max')
            ES = callbacks.EarlyStopping(monitor='val_auc', patience=10, verbose=verbose_ckp, mode='max')
        else:
            MCP = callbacks.ModelCheckpoint(path, monitor='val_loss', verbose=verbose_ckp, save_best_only=True, mode='min') 
            ES = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=verbose_ckp, mode='min')
        train_gen = self.generator(train_hadm, batchSize)
        valid_gen = self.generator(valid_hadm, batchSize)
        self.model.fit(train_gen, validation_data=valid_gen, epochs=epochs, verbose=verbose_fit, callbacks=[MCP, ES], 
                       steps_per_epoch=len(train_hadm)//batchSize, validation_steps=len(valid_hadm)//batchSize)
        self.model = models.load_model(path, compile=False)

    def load(self, path):
        self.model = models.load_model(path, compile=False)
        
    def predict(self, test_hadm, batchSize, verbose=2):
        pred_gen = self.pred_generator(test_hadm, batchSize)
        pred = self.model.predict(pred_gen, verbose=verbose, steps=(len(test_hadm)+batchSize-1)//batchSize)
        return pred

    def generator(self, input_hadm, batchSize):
        while 1:
            hadm_ids = np.sort(np.random.choice(input_hadm, batchSize, False))
            x = self.get_x(hadm_ids, batchSize)
            y = self.get_y(hadm_ids)
            yield x, y
            
    def pred_generator(self, input_hadm, batchSize):
        i = 0
        while 1:
            hadm_ids = input_hadm[np.arange(i, min(i+batchSize, len(input_hadm)))]
            if i+batchSize >= len(input_hadm):
                i = 0
            i += len(hadm_ids)
            x = self.get_x(hadm_ids, len(hadm_ids))
            y = self.get_y(hadm_ids)
            yield x, y
            
    def get_y(self, hadm_ids):
        batch_cohort = self.cohort[self.cohort.hadm_id.isin(hadm_ids)].sort_values('hadm_id')
        y = batch_cohort[self.label].to_numpy()
        return y

    def get_input_shapes(self):
        xs = []
        xd = ( 3, 306)
        xl = (14, 107)
        xp = ( 2,  18)
        xm = (14,  55)
    
        if self.use_diag:
            xs.append(xd)

        if self.use_labs:
            xs.append(xl)

        if self.use_proc:
            xs.append(xp)
            
        if self.use_meds:
            xs.append(xm)
            
        cohort_columns = []
        if self.use_insurance:
            cohort_columns.append('medicare')
            cohort_columns.append('medicaid')
        if self.use_gender:
            cohort_columns.append('male')
        if self.use_race:
            cohort_columns.append('white')
        if len(cohort_columns) > 0:
            xs.append((len(cohort_columns),))

        return xs
    
    def get_x(self, hadm_ids, batchSize):
        batch_cohort = self.cohort[self.cohort.hadm_id.isin(hadm_ids)].sort_values('hadm_id')

        xs = []
        xd = np.zeros((batchSize,  3, 306))
        xl = np.zeros((batchSize, 14, 107))
        xp = np.zeros((batchSize,  2,  18))
        xm = np.zeros((batchSize, 14,  55))

        if self.use_diag:
            for i in range(batchSize):
                seqnum = min(xd.shape[1], batch_cohort.seqnum.iloc[i]+1)
                raw_index = batch_cohort.index[i]
                xd[i, 0:seqnum] = self.diag.loc[self.cohort[(raw_index-seqnum+1):(raw_index+1)].hadm_id.to_numpy()].to_numpy()
            xs.append(xd)

        if self.use_labs:
            for i in range(batchSize):
                hadm_id = hadm_ids[i]
                if hadm_id in self.labs.index:
                    labs_i = self.labs.loc[hadm_id:hadm_id+1]
                    seqnum = min(xl.shape[1], len(labs_i))
                    xl[i, 0:seqnum] = labs_i.iloc[-seqnum:].to_numpy()
            xs.append(xl)

        if self.use_proc:
            for i in range(batchSize):
                hadm_id = hadm_ids[i]
                if hadm_id in self.proc.index:
                    proc_i = self.proc.loc[hadm_id:hadm_id+1]
                    seqnum = min(xp.shape[1], len(proc_i))
                    xp[i, 0:seqnum] = proc_i.iloc[-seqnum:].to_numpy()
            xs.append(xp)

        if self.use_meds:
            for i in range(batchSize):
                hadm_id = hadm_ids[i]
                if hadm_id in self.meds.index:
                    meds_i = self.meds.loc[hadm_id:hadm_id+1]
                    seqnum = min(xm.shape[1], len(meds_i))
                    xm[i, 0:seqnum] = meds_i.iloc[-seqnum:].to_numpy()
            xs.append(xm)

        cohort_columns = []
        if self.use_insurance:
            cohort_columns.append('medicare')
            cohort_columns.append('medicaid')
        if self.use_gender:
            cohort_columns.append('male')
        if self.use_race:
            cohort_columns.append('white')
        if len(cohort_columns) > 0:
            xs.append(batch_cohort[cohort_columns].to_numpy())

        return xs

# All

In [52]:
# Settings
use_diag = 1
use_labs = 1
use_proc = 1
use_meds = 1
use_insurance = 1
use_gender = 1
use_race = 1
hf = 0
lr = 1e-3
epochs = 100
batchSize = 256
path='./models/all.h5'

# Initialize Model
model, train_hadm, valid_hadm, test_hadm = initialize(lr, cohort, diag, labs, proc, meds, use_diag, use_labs, use_proc, use_meds, use_insurance, use_gender, use_race, hf)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 3, 306)]             0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 14, 107)]            0         []                            
                                                                                                  
 input_3 (InputLayer)        [(None, 2, 18)]              0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 14, 55)]             0         []                            
                                                                                              

In [53]:
# Train
model.train(train_hadm, valid_hadm, epochs, batchSize, path, False, 1, 1)

Epoch 1/100
91/91 [==============================] - ETA: 0s - loss: 0.4769 - auc: 0.6642
Epoch 1: val_loss improved from inf to 0.44444, saving model to ./models/all.h5
91/91 [==============================] - 160s 2s/step - loss: 0.4769 - auc: 0.6642 - val_loss: 0.4444 - val_auc: 0.7119
Epoch 2/100


/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


91/91 [==============================] - ETA: 0s - loss: 0.4223 - auc: 0.7448
Epoch 2: val_loss improved from 0.44444 to 0.44168, saving model to ./models/all.h5
91/91 [==============================] - 154s 2s/step - loss: 0.4223 - auc: 0.7448 - val_loss: 0.4417 - val_auc: 0.7294
Epoch 3/100


/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


91/91 [==============================] - ETA: 0s - loss: 0.4209 - auc: 0.7535
Epoch 3: val_loss improved from 0.44168 to 0.42882, saving model to ./models/all.h5
91/91 [==============================] - 160s 2s/step - loss: 0.4209 - auc: 0.7535 - val_loss: 0.4288 - val_auc: 0.7327
Epoch 4/100


/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


91/91 [==============================] - ETA: 0s - loss: 0.4138 - auc: 0.7696
Epoch 4: val_loss did not improve from 0.42882
91/91 [==============================] - 169s 2s/step - loss: 0.4138 - auc: 0.7696 - val_loss: 0.4511 - val_auc: 0.7095
Epoch 5/100
91/91 [==============================] - ETA: 0s - loss: 0.3911 - auc: 0.7948
Epoch 5: val_loss did not improve from 0.42882
91/91 [==============================] - 210s 2s/step - loss: 0.3911 - auc: 0.7948 - val_loss: 0.4407 - val_auc: 0.7070
Epoch 6/100
91/91 [==============================] - ETA: 0s - loss: 0.3730 - auc: 0.8107
Epoch 6: val_loss did not improve from 0.42882
91/91 [==============================] - 169s 2s/step - loss: 0.3730 - auc: 0.8107 - val_loss: 0.4433 - val_auc: 0.7265
Epoch 7/100
91/91 [==============================] - ETA: 0s - loss: 0.3698 - auc: 0.8202
Epoch 7: val_loss did not improve from 0.42882
91/91 [==============================] - 193s 2s/step - loss: 0.3698 - auc: 0.8202 - val_loss: 0.4655 - 

In [60]:
def fairness(bias, m, y_true, res):
    #y_true = cohort.set_index('hadm_id').loc[m]['label_diabetes'].to_numpy()
    #pred = model.predict(train_hadm, batchSize, 1)
  
    bias_labels = []
    for hadm_id in m:
        patient_data = cohort.loc[cohort['hadm_id'] == hadm_id]
        bias_labels.append(patient_data[bias].iloc[0])  

    # use mean has threshold
    threshold = np.mean(res)

    # convert res into binary prediction
    binary_pred = [1 if x > threshold else 0 for x in res]

    eod_diff= equalized_odds_difference(y_true, binary_pred, sensitive_features=bias_labels)
    eod_ratio = equalized_odds_ratio(y_true, binary_pred, sensitive_features=bias_labels)

    return eod_diff, eod_ratio

In [ ]:
# Training Results
model.load(path)
res = model.predict(train_hadm, batchSize, 1)
y_true = cohort.set_index('hadm_id').loc[train_hadm].label_diabetes.to_numpy()
auc=roc_auc_score(y_true, res)
prec=average_precision_score(y_true, res)
print(f'AUC Score: {auc}')
print(f'AUPCR Score: {prec}')

m='train_hadm'
race='white'
diff, ratio=fairness(race, m, y_true, res)
print("Equalized Odds Difference (Race Parity):", diff)
print("Equalized Odds Ratio (Race Parity):", ratio)

gender='male'
diff, ratio=fairness(gender, m,  y_true, res) 
print("Equalized Odds Difference (Gender Parity):", diff)
print("Equalized Odds Ratio (Gender Parity):", ratio)

insurance='medicaid'
diff, ratio=fairness(insurance, m,  y_true, res)
print("Equalized Odds Difference (Medicaid Parity):", diff)
print("Equalized Odds Ratio (Medicaid Parity):", ratio)

insurance='medicare'
diff, ratio=fairness(insurance, m,  y_true, res)
print("Equalized Odds Difference (Medicare Parity):", diff)
print("Equalized Odds Ratio (Medicare Parity):", ratio)


In [61]:
# Validation Results
model.load(path)
res = model.predict(valid_hadm, batchSize, 1)
y_true = cohort.set_index('hadm_id').loc[valid_hadm].label_diabetes.to_numpy()
roc_auc_score(y_true, res), average_precision_score(y_true, res)

auc=roc_auc_score(y_true, res)
prec=average_precision_score(y_true, res)
print(f'AUC Score: {auc}')
print(f'AUPCR Score: {prec}')

race='white'
m=valid_hadm
diff, ratio=fairness(race, m,  y_true, res)
print("Equalized Odds Difference (Race Parity):", diff)
print("Equalized Odds Ratio (Race Parity):", ratio)

gender='male'
diff, ratio=fairness(gender, m , y_true, res) 
print("Equalized Odds Difference (Gender Parity):", diff)
print("Equalized Odds Ratio (Gender Parity):", ratio)

insurance='medicaid'
diff, ratio=fairness(insurance, m,  y_true, res)
print("Equalized Odds Difference (Medicaid Parity):", diff)
print("Equalized Odds Ratio (Medicaid Parity):", ratio)

insurance='medicare'
diff, ratio=fairness(insurance, m,  y_true, res)
print("Equalized Odds Difference (Medicare Parity):", diff)
print("Equalized Odds Ratio (Medicare Parity):", ratio)

31/31 [==============================] - 37s 1s/step
AUC Score: 0.7199627511569392
AUPCR Score: 0.37025478819726676
Equalized Odds Difference (Race Parity): 0.11650234472901838
Equalized Odds Ratio (Race Parity): 0.7407183345851188
Equalized Odds Difference (Gender Parity): 0.07309971749666877
Equalized Odds Ratio (Gender Parity): 0.8117369639588127
Equalized Odds Difference (Medicaid Parity): 0.14152220189059628
Equalized Odds Ratio (Medicaid Parity): 0.7104736633077878
Equalized Odds Difference (Medicare Parity): 0.0500956558256731
Equalized Odds Ratio (Medicare Parity): 0.927552466428234


In [ ]:
# Test Results
model.load(path)
res = model.predict(test_hadm, batchSize, 1)
y_true = cohort.set_index('hadm_id').loc[test_hadm].label_diabetes.to_numpy()
roc_auc_score(y_true, res), average_precision_score(y_true, res)

auc=roc_auc_score(y_true, res)
prec=average_precision_score(y_true, res)
print(f'AUC Score: {auc}')
print(f'AUPCR Score: {prec}')

race='white'
m='test_hadm'
diff, ratio=fairness(race, m,  y_true, res)
print("Equalized Odds Difference (Race Parity):", diff)
print("Equalized Odds Ratio (Race Parity):", ratio)

gender='male'
diff, ratio=fairness(gender, m , y_true, res) 
print("Equalized Odds Difference (Gender Parity):", diff)
print("Equalized Odds Ratio (Gender Parity):", ratio)

insurance='medicaid'
diff, ratio=fairness(insurance, m,  y_true, res)
print("Equalized Odds Difference (Medicaid Parity):", diff)
print("Equalized Odds Ratio (Medicaid Parity):", ratio)

insurance='medicare'
diff, ratio=fairness(insurance, m,  y_true, res)
print("Equalized Odds Difference (Medicare Parity):", diff)
print("Equalized Odds Ratio (Medicare Parity):", ratio)

31/31 [==============================] - 31s 1s/step
AUC Score: 0.7173296475013702
AUPCR Score: 0.36429435408782884
